In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#pip install sentencepiece transformers

In [ ]:
import re
import sys
import typing as tp
import unicodedata
from sacremoses import MosesPunctNormalizer


mpn = MosesPunctNormalizer(lang="en")
mpn.substitutions = [
    (re.compile(r), sub) for r, sub in mpn.substitutions
]


def get_non_printing_char_replacer(replace_by: str = " ") -> tp.Callable[[str], str]:
    non_printable_map = {
        ord(c): replace_by
        for c in (chr(i) for i in range(sys.maxunicode + 1))
        if unicodedata.category(c) in {"C", "Cc", "Cf", "Cs", "Co", "Cn"}
    }

    def replace_non_printing_char(line) -> str:
        return line.translate(non_printable_map)

    return replace_non_printing_char


replace_nonprint = get_non_printing_char_replacer(" ")


def preproc(text):
    clean = mpn.normalize(text)
    clean = replace_nonprint(clean)
    clean = unicodedata.normalize("NFKC", clean)
    return clean

In [ ]:
#DATA_PATH = 'C://Users//Andrej//1_optimiz//'
df = pd.read_csv(DATA_PATH + 'data.csv')

In [ ]:
df.target = df.target.apply(preproc)

In [ ]:
df.source = df.source.apply(preproc)

In [ ]:
df.head()

,Unnamed: 0,target,source
0,0,Та пыгрисит маим вармаль э̄рнэ поратэт ат верм...,Те мальчики не выполнят задание в назначенный ...
1,1,"Ха̄йтыматэ тӯр ва̄тан ёхтыс, вит ва̄тан ха̄йтыс.","Бегая к берегу озера пришла, к воде подбежала."
2,2,Вит са̄мыл сунсым о̄нтыс,Вода прибывала на глазах
3,3,"Атаявев, акваг лылынг тагл ворн та тотавев.","Обнюхивает нас, живыми на кладбище уносит."
4,4,"Ман ты пӣлтал, веськат хумиюв нэтхуньт ат ёрув...",Мы никогда не забудем этого честного человека.


In [ ]:
len(df)

81146

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train, df_test_split = train_test_split(df, test_size=0.1, random_state=121)

In [ ]:
df_test, df_validation = train_test_split(df_test_split, test_size=0.8, random_state=121)

In [ ]:
len(df_test)

1623

# Evaluation

In [ ]:
import os
from transformers import AutoModelForSeq2SeqLM, NllbTokenizer

MODEL_SAVE_PATH = '/content/drive/MyDrive/Colab Notebooks/Hackathon/Trained_models'

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_SAVE_PATH).cuda();
tokenizer = NllbTokenizer.from_pretrained(MODEL_SAVE_PATH)

In [ ]:
def translate(
    text, src_lang='rus_Cyrl', tgt_lang='eng_Latn',
    a=32, b=3, max_input_length=1024, num_beams=4, **kwargs
):
    """Turn a text or a list of texts into a list of translations"""
    tokenizer.src_lang = src_lang
    tokenizer.tgt_lang = tgt_lang
    inputs = tokenizer(
        text, return_tensors='pt', padding=True, truncation=True,
        max_length=max_input_length
    )
    model.eval() # turn off training mode
    result = model.generate(
        **inputs.to(model.device),
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang),
        max_new_tokens=int(a + b * inputs.input_ids.shape[1]),
        num_beams=num_beams, **kwargs
    )
    return tokenizer.batch_decode(result, skip_special_tokens=True)

In [ ]:
# Example usage:
t = 'А̄лы ма̄т о̄лнэ то̄ва ӯйхулт вит та̄нти палтыланыл ӯньщим о̄луӈкв ханьщувласт,  таи ма̄гсыл хо̄са вит тал о̄луӈкв вēрмегы.'
print(translate(t, 'mns_Cyrl', 'rus_Cyrl'))
# ['Обитатели южных районов научились накапливать в организме запасы влаги, по этому по долгу могут обходиться без воды.']

['Обитатели южных районов научились накапливать в организме запасы влаги, по этому по долгу могут обходиться без воды.']


In [ ]:
# Example usage:
t = 'Его много народу знает, во многих странах известен.'
print(translate(t, 'rus_Cyrl', 'mns_Cyrl'))
# ['Тав са̄в хо̄тпан ва̄ве, са̄в ма̄т ва̄ве.']

['Тав са̄в хо̄тпан ва̄ве, са̄в ма̄т ва̄ве.']


In [ ]:
def batched_translate(texts, batch_size=16, **kwargs):
    """Translate texts in batches of similar length"""
    idxs, texts2 = zip(*sorted(enumerate(texts), key=lambda p: len(p[1]), reverse=True))
    results = []
    for i in trange(0, len(texts2), batch_size):
        results.extend(translate(texts2[i: i+batch_size], **kwargs))
    return [p for i, p in sorted(zip(idxs, results))]

In [ ]:
from tqdm.auto import tqdm

In [ ]:
df_test['source_translated'] = [translate(t, 'mns_Cyrl', 'rus_Cyrl')[0] for t in tqdm(df_test.target)]
df_test['target_translated'] = [translate(t, 'rus_Cyrl', 'mns_Cyrl')[0] for t in tqdm(df_test.source)]

  0%|          | 0/1623 [00:00<?, ?it/s]

  0%|          | 0/1623 [00:00<?, ?it/s]

In [ ]:
import sacrebleu
bleu_calc = sacrebleu.BLEU(max_ngram_order=4)
chrf_calc = sacrebleu.CHRF(word_order=2)  # this metric is called ChrF++   word_order=2

# 0.240 text preproc - both

print(bleu_calc.corpus_score(df_test['source_translated'].tolist(), [df_test['source'].tolist()]))
print(chrf_calc.corpus_score(df_test['source_translated'].tolist(), [df_test['source'].tolist()]))
print(bleu_calc.corpus_score(df_test['target_translated'].tolist(), [df_test['target'].tolist()]))
print(chrf_calc.corpus_score(df_test['target_translated'].tolist(), [df_test['target'].tolist()]))

BLEU = 24.97 57.3/29.4/18.5/12.5 (BP = 1.000 ratio = 1.000 hyp_len = 16658 ref_len = 16659)
chrF2++ = 49.07
BLEU = 27.06 57.5/32.2/21.5/15.4 (BP = 0.966 ratio = 0.967 hyp_len = 15608 ref_len = 16145)
chrF2++ = 53.04


In [ ]:
df_test

,Unnamed: 0,target,source,source_translated,target_translated
1021,1021,Тамле мо̄йт ты о̄лы.,Вот такая сказка есть.,Вот такая эта сказка.,Тамле ты мо̄йт о̄лы.
69803,17966,Яныг а̄гитэ̄н Ольга Ня̄хщамво̄ль па̄вылт Дима ...,Старшая дочь Ольга с сыном Димой живут в посёл...,Старшая дочь Ольга в Няксимволе с сыном Димой ...,Яныг а̄гитэ̄н Ольга Дима пыге ёт Ня̄хщамво̄ль ...
46160,46160,Тот та сунсыглахтуӈкве патыс.,Там он начал осматриваться.,И там начала рассматривать всё.,Тув сунсыглахтуӈкве патыс.
30347,30347,"Нила ургалахтым ма̄хум та̄нти хот-рохтысыт, то...",Четверо стражников от страха перед ним затрясл...,"Четыре берестяных человека сами испугались, др...","Нила урнэ хум тав э̄лы-пāлэт хот-рохтысыт, хол..."
40354,40354,"Товлыӈхап нё̄лэт ё̄л ке пēлхаты, на̄н са̄квата...","Если самолёт ударится носом, убьётесь.","Если самолет сгорит с носа, вы можете раниться.","Товлыӈха̄пын нёл ке хо̄йи, алэ̄н."
...,...,...,...,...,...
2071,2071,"Э̄рттам, лё̄муйт лё̄муй-а̄щаныл па̄лт ювле ёхт...","Будто бы, все комары возвращаются к своему отц...",Будто бы комары к отцу-комару обратно придут.,"Аквтопмат ань лё̄муйт ань, акв ювле лё̄муй а̄щ..."
42492,42492,"Са̄сныл матыр-а̄ти щаквщуӈкве нас хунь, хорамы...","Работать с берестой - это, конечно, непростое ...","Из бересты делать что не просто, для рисования...","Са̄с ёт рӯпитаӈкве - тыи, ва̄им, нас рӯпата ху..."
43209,43209,"ма̄ньщиянув сахил, мольщаӈыл, ва̄ил манос ня̄р...",Взрослые и дети одеты в красивые зимние одежды...,"Мансийских людей в шубе, малице, в меховой обу...",Яныг хо̄тпат ос ня̄врамыт тэ̄лы са̄всыр нё̄тнэ...
8818,8818,Тав та̄лэ ат ро̄ви ты тэ̄лан вуйхатуӈкв,Без него нельзя браться за это дело,Без него нельзя делать это дело.,Тав та̄лэ ат тайи вуйхатахтуӈкв
